We first get necessary external data and code

In [82]:
!git clone https://github.com/AllenInstitute/deepinterpolation.git
!mkdir -p ephys

fatal: destination path 'deepinterpolation' already exists and is not an empty directory.


Install deepinterpolation package

In [83]:
!pip install git+https://github.com/AllenInstitute/deepinterpolation.git

  Cloning https://github.com/AllenInstitute/deepinterpolation.git to /tmp/pip-req-build-5qf0jam7
  Running command git clone -q https://github.com/AllenInstitute/deepinterpolation.git /tmp/pip-req-build-5qf0jam7
  Created wheel for deepinterpolation: filename=deepinterpolation-0.1.0-cp36-none-any.whl size=21726 sha256=ffeedf4fed39bc8d211a523b1cb6244ebaf13ba60b85d8023e4c8e98bd9265fb
  Stored in directory: /tmp/pip-ephem-wheel-cache-uf59wkv4/wheels/f1/78/fd/1d2e8acccfd7c204055c5ab9dbc5add837981811641c01dbf1
Successfully built deepinterpolation


In [84]:
import deepinterpolation as de
import sys
from shutil import copyfile
import os
from deepinterpolation.generic import JsonSaver, ClassLoader
import datetime
from typing import Any, Dict
import pathlib
import sys

This is used for record-keeping


In [85]:
now = datetime.datetime.now()
run_uid = now.strftime("%Y_%m_%d_%H_%M")

Initialize meta-parameters objects

In [86]:
training_param = {}
generator_param = {}
network_param = {}
generator_test_param = {}

An epoch is defined as the number of batches pulled from the dataset. Because our datasets are VERY large. Often, we cannot
go through the entirity of the data so we define an epoch slightly differently than is usual.

In [87]:
steps_per_epoch = 10

Those are parameters used for the Validation test generator. Here the test is done on the beginning of the data but
this can be a separate file

In [88]:
generator_test_param["type"] = "generator"  # type of collection
generator_test_param["name"] = "EphysGenerator"  # Name of object in the collection
generator_test_param[
    "pre_post_frame"
] = 30  # Number of frame provided before and after the predicted frame
generator_test_param["train_path"] = os.path.join(
    "deepinterpolation",
    "sample_data",
    "ephys_tiny_continuous.dat2",
)
generator_test_param["batch_size"] = 100
generator_test_param["start_frame"] = 0
generator_test_param["end_frame"] = 1999
generator_test_param[
    "pre_post_omission"
] = 1  # Number of frame omitted before and after the predicted frame
generator_test_param["steps_per_epoch"] = -1  # No step necessary for testing as epochs are not relevant. -1 deactivate it.

Those are parameters used for the main data generator

In [89]:
generator_param["type"] = "generator"
generator_param["steps_per_epoch"] = steps_per_epoch
generator_param["name"] = "EphysGenerator"
generator_param["pre_post_frame"] = 30
generator_param["train_path"] = os.path.join(
    "deepinterpolation",
    "sample_data",
    "ephys_tiny_continuous.dat2",
)
generator_param["batch_size"] = 100
generator_param["start_frame"] = 2000
generator_param["end_frame"] = 7099
generator_param["pre_post_omission"] = 1

Those are parameters used for the network topology

In [90]:
network_param["type"] = "network"
network_param[
    "name"
] = "unet_single_ephys_1024"  # Name of network topology in the collection

Those are parameters used for the training process

In [91]:
training_param["type"] = "trainer"
training_param["name"] = "core_trainer"
training_param["run_uid"] = run_uid
training_param["batch_size"] = generator_test_param["batch_size"]
training_param["steps_per_epoch"] = steps_per_epoch
training_param[
    "period_save"
] = 25  # network model is potentially saved during training between a regular nb epochs
training_param["nb_gpus"] = 0
training_param["apply_learning_decay"] = 0
training_param[
    "nb_times_through_data"
] = 1  # if you want to cycle through the entire data. Two many iterations will cause noise overfitting
training_param["learning_rate"] = 0.0001
training_param["pre_post_frame"] = generator_test_param["pre_post_frame"]
training_param["loss"] = "mean_absolute_error"
training_param[
    "nb_workers"
] = 1  # this is to enable multiple threads for data generator loading. Useful when this is slower than training

training_param["model_string"] = (
    network_param["name"]
    + "_"
    + training_param["loss"]
    + "_"
    + training_param["run_uid"]
)

Where do you store ongoing training progress

In [92]:
 jobdir = os.path.join(
    "ephys", training_param["model_string"] + "_" + run_uid,
)
training_param["output_dir"] = jobdir

try:
    os.mkdir(jobdir)
except:
    print("folder already exists")

Here we create all json files that are fed to the training. This is used for recording purposes as well as input to the training proces

In [93]:
path_training = os.path.join(jobdir, "training.json")
json_obj = JsonSaver(training_param)
json_obj.save_json(path_training)

path_generator = os.path.join(jobdir, "generator.json")
json_obj = JsonSaver(generator_param)
json_obj.save_json(path_generator)

path_test_generator = os.path.join(jobdir, "test_generator.json")
json_obj = JsonSaver(generator_test_param)
json_obj.save_json(path_test_generator)

path_network = os.path.join(jobdir, "network.json")
json_obj = JsonSaver(network_param)
json_obj.save_json(path_network)

Here we create all objects for training.

In [94]:
# We find the generator obj in the collection using the json file
generator_obj = ClassLoader(path_generator)
generator_test_obj = ClassLoader(path_test_generator)

# We find the network obj in the collection using the json file
network_obj = ClassLoader(path_network)

# We find the training obj in the collection using the json file
trainer_obj = ClassLoader(path_training)

# We build the generators object. This will, among other things, calculate normalizing parameters.
train_generator = generator_obj.find_and_build()(path_generator)
test_generator = generator_test_obj.find_and_build()(path_test_generator)

# We build the network object. This will, among other things, calculate normalizing parameters.
network_callback = network_obj.find_and_build()(path_network)

# We build the training object.
training_class = trainer_obj.find_and_build()(train_generator, test_generator, network_callback, path_training)

Start training. This can take very long time.

In [95]:
training_class.run()

Epoch 1/5
10/10 [==============================] - 67s 7s/step - loss: 0.4588 - val_loss: 0.3927
Epoch 2/5
10/10 [==============================] - 67s 7s/step - loss: 0.3837 - val_loss: 0.3892
Epoch 3/5
10/10 [==============================] - 67s 7s/step - loss: 0.3703 - val_loss: 0.3773
Epoch 4/5
10/10 [==============================] - 66s 7s/step - loss: 0.3827 - val_loss: 0.3716
Epoch 5/5
10/10 [==============================] - 66s 7s/step - loss: 0.3846 - val_loss: 0.3677


Finalize and save output of the training.

In [96]:
training_class.finalize()

Saved model to disk
